<a href="https://colab.research.google.com/github/philosophynote/machine_learning/blob/main/agent_ch9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.0
    Uninstalling langsmith-0.3.0:
      Successfully uninstalled langsmith-0.3.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [4]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

In [6]:
import operator

from typing import Annotated
from langchain_core.pydantic_v1 import BaseModel, Field

In [7]:
class State(BaseModel):
    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(
        default="", description="選定された回答ロール"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴"
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

In [11]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [12]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [41]:
def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}



In [29]:
def answer_node(state: State) -> dict[str, Any]:
  query = state.query
  role = state.current_role
  role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
  prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
    )
  chain = prompt | llm | StrOutputParser()
  answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
  return {"messages":  [answer]}

In [16]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="判定結果")
    reason: str = Field(default="", description="判定理由")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [42]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [43]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answer_node)
workflow.add_node("check", check_node)

In [44]:
workflow.set_entry_point("selection")

In [45]:
workflow.add_edge("selection", "answering")
workflow.add_edge("answering", "check")

In [46]:
from langgraph.graph import END

In [47]:
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [48]:
compiled = workflow.compile()

In [51]:
initial_state = State(query="子供を産むことは人類にとって幸せなのでしょうか？")
result = compiled.invoke(initial_state)

In [52]:
result

{'query': '子供を産むことは人類にとって幸せなのでしょうか？',
 'current_role': 'カウンセラー',
 'messages': ['この質問は非常に個人的で、答えは人それぞれ異なるかもしれません。子供を持つことが幸せかどうかは、個人の価値観、ライフスタイル、人生の目標によって大きく変わります。\n\n多くの人にとって、子供を持つことは大きな喜びや充実感をもたらします。子供の成長を見守り、家族としての絆を深めることは、人生において非常に意義深い経験となることが多いです。また、子供を育てることで得られる学びや成長も、親にとって大きな財産となるでしょう。\n\n一方で、子供を持つことには多くの責任や挑戦が伴います。育児には時間、エネルギー、経済的な負担がかかるため、これらを考慮に入れた上での決断が必要です。子供を持たない選択をする人も増えており、それもまた一つの幸せの形です。\n\n最終的には、自分自身の価値観や人生の目標に基づいて、どのような選択が自分にとって最も幸せなのかを考えることが大切です。もしこのテーマについてさらに深く考えたい場合や、具体的な悩みがある場合は、信頼できる人や専門家に相談することも一つの方法です。あなたの選択が、あなた自身にとって最も満足のいくものであることを願っています。'],
 'current_judge': True,
 'judgement_reason': '回答は非常にバランスが取れており、個人の価値観や状況に応じた多様な視点を提供しています。子供を持つことの利点と挑戦の両方を公平に説明し、最終的な決断は個人の価値観に基づくべきであると強調しています。また、専門家への相談を勧めることで、より深い理解を促しています。全体として、質問に対する適切で包括的な回答です。'}